In [74]:
import ollama
import json
import os
from tqdm import tqdm

## Preprocess json file if it does not exist

In [75]:
origninal_file_name = 'combined_max_short.json'
file_name = "new_combined_max_short.json"
file_exists = os.path.exists(f"./{file_name}")

In [76]:
if not file_exists:
    with open(origninal_file_name, 'r') as file:
        data = file.read()
    new_data = data.replace("}}\n", "}},\n")
    new_data = "[" + new_data
    new_data = new_data[:-2] + "]"
    with open(file_name, "w") as text_file:
        text_file.write(new_data)

## Read json file

In [77]:
with open(file_name) as f:
    text_collection = json.load(f)
n = len(text_collection)

## Rephrasing

### Rephrasing styles

In [102]:
easy_style = "For the following paragraph give me only one paraphrase of the same using a very small vocabulary and extremely simple sentences that a toddler will understand:\n"
medium_style = "For the following paragraph give me only one diverse paraphrase of the same in high quality English language as in sentences on Wikipedia:\n"
hard_style = "For the following paragraph give me only one paraphrase of the same using very terse and abstruse language that only an erudite scholar will understand. Replace simple words and phrases with rare and complex ones:\n"

### Set current style

In [103]:
styles = {'medium_rephrase': medium_style}

### Sort texts by length

In [80]:
for i in tqdm(range(n)):
    l = len(text_collection[i]['text'])
    text_collection[i]['length'] = l

100%|██████████| 213/213 [00:00<00:00, 255326.31it/s]


In [81]:
text_collection_sorted = sorted(text_collection, key=lambda d: d['length'])

### Do rephrase

In [106]:
for style_desc in styles:
    style = styles[style_desc]
    for i in tqdm(range(n)):
        response = ollama.chat(model='mistral:instruct', messages=[
            {
                'role': 'user',
                'content': f"Paragraph: {text_collection_sorted[i]['text']}\n{style}",
            }
        ])
        rephrasing = response['message']['content']
        rephrasing = rephrasing.replace("\n\n", "\n")
        text_collection_sorted[i][style_desc] = rephrasing

100%|██████████| 2/2 [01:37<00:00, 48.78s/it]


### Save rephrase

In [113]:
with open(f'./rephrased_{origninal_file_name}.json', 'w') as fout:
    json.dump(text_collection_sorted , fout, indent=2)